# "European Covid data exploration"
> "Exploring which countries have had the highest and lowest covid numbers in Europe"

- toc: true
- branch: master
- badges: true
- comments: false
- author: Ifan Johnston
- categories: [covid]

# Importing and preparing the data

We will be looking at data from the following countries:

- Ukraine
- Austria
- Germany
- Belgium
- Sweden
- United Kingdom

We begin by importing the data, and adding calculating some new features so that we can compare the data from different countries. For example we calculate 'confirmed cases per 100k population', 'deaths per 100k' and 'new cases' since these are not initially in the dataset.

In [1]:
#collapse
from covid19dh import covid19
import altair as alt
import datetime

countries = ["Ukraine", 
             "Austria",
             "Germany",
             "Belgium",
             "Sweden",
             "United Kingdom"
            ]

yesterday = datetime.date.today() - datetime.timedelta(days=1)

x, src = covid19(countries, raw=False, verbose=False, end=yesterday)

x_small = x.loc[:, ['administrative_area_level_1', 'date', 'vaccines', 'confirmed','tests', 'recovered', 'deaths', 'population']]
x_small.rename(columns={'administrative_area_level_1': 'id'}, inplace=True)

x_small['confirmed_per'] = 100000 * x_small['confirmed'] / x_small['population']
x_small['deaths_per'] = 100000 * x_small['deaths'] / x_small['population']
x_small['ratio'] = 100 * (x_small['deaths']) / (x_small['confirmed'])
x_small['tests_per'] = 100000 * (x_small['tests']) / (x_small['population'])

x_small['new_cases']=x_small.groupby('id').confirmed.diff().fillna(0)
x_small['new_cases_per']=x_small.groupby('id').confirmed_per.diff().fillna(0)

Here is a random sample of 5 rows from the dataset.

In [2]:
x_small.sample(5)

,id,date,vaccines,confirmed,tests,recovered,deaths,population,confirmed_per,deaths_per,ratio,tests_per,new_cases,new_cases_per
20306,Germany,2020-05-23,0.0,178786,3529962,169766,9019,82905782,215.649615,10.878614,5.044578,4257.799534,338.0,0.407692
20494,Germany,2020-11-27,0.0,1039338,28111640,1016058,23262,82905782,1253.637533,28.058357,2.238155,33907.936602,20154.0,24.309523
27685,United Kingdom,2020-08-17,0.0,319197,14825051,344,41369,66460344,480.281896,62.246142,12.960335,22306.611895,713.0,1.072820
72742,Sweden,2020-08-29,0.0,84189,0,0,5786,10175214,827.392918,56.863669,6.872632,0.000000,131.0,1.287442
4482,Austria,2020-07-11,0.0,18821,691709,16771,738,8840521,212.894693,8.347924,3.921152,7824.301305,91.0,1.029351


# Plotting the data

We will first look at the total numbers of cases and deaths in each country, before moving on to cases and deaths per 100k population.

{% include info.html text="In each of the charts below, you can click on the legend to filter the lines shown" %}

## Total cases per 100,000

In [3]:
#collapse

leg_selection = alt.selection_multi(fields=['id'], bind='legend')

alt.Chart(x_small).mark_line().encode(
    x=alt.X("yearmonthdate(date):T", axis=alt.Axis(title='Date')),
    y=alt.Y("confirmed_per:Q", axis=alt.Axis(title='Confirmed per 100k')),
    tooltip='id',
    color=alt.Color('id', legend=alt.Legend(title="Countries")),
    opacity=alt.condition(leg_selection, alt.value(1), alt.value(0.2))
).add_selection(leg_selection).properties(title='Total number of cases per 100,000 population for selected European Countries', width=600).interactive()


alt.Chart(...)

## Total deaths per 100,000

In [4]:
#collapse
alt.Chart(x_small).mark_line().encode(
    x=alt.X("yearmonthdate(date):T", axis=alt.Axis(title='Date')),
    y=alt.Y("deaths_per:Q", axis=alt.Axis(title='Deaths per 100k')),
    tooltip='id',
    color=alt.Color('id', legend=alt.Legend(title="Countries")),
    opacity=alt.condition(leg_selection, alt.value(1), alt.value(0.2))
).add_selection(leg_selection).properties(title='Number of deaths per 100,000 population for selected European Countries', width=600).interactive()

alt.Chart(...)

## New cases per 100,000

In [5]:
#collapse
brush = alt.selection(type='interval', encodings=['x'])

base = alt.Chart(x_small).mark_line().transform_window(
    rolling_mean='sum(new_cases_per)',
    frame=[-7, 0],
    groupby=['id']
).encode(
    x=alt.X("yearmonthdate(date):T",
            axis=alt.Axis(title='Date')
           ),
    y=alt.Y("rolling_mean:Q",
            axis=alt.Axis(title='Incidence rate')
           ),
    tooltip='id',
    color=alt.Color('id', legend=alt.Legend(title="Countries")),
    opacity=alt.condition(leg_selection, alt.value(1), alt.value(0.2))
).add_selection(leg_selection).properties(
    width=600,
    height=400,
    title='Number of new cases per 100,000 for selected countries'
)

upper = base.encode(
    alt.X('yearmonthdate(date):T',axis=alt.Axis(title='Date'),
          scale=alt.Scale(domain=brush))
)

lower = base.properties(
    height=60
).add_selection(brush)

upper & lower

alt.VConcatChart(...)

The ratio of confirmed cases and deaths gives an indication of what the case fatality rate is - it seems to be between 2 and 3%, assuming that the countries listed here are catching all positive cases (which they probably aren't, so it's likely lower than this).

## Case fatality rate

In [6]:
#collapse
base = alt.Chart(x_small).mark_line().encode(
    x=alt.X("yearmonthdate(date):T", axis=alt.Axis(title='Date')),
    y=alt.Y("ratio:Q", axis=alt.Axis(title='Ratio of deaths per case')),
    tooltip='id',
    color=alt.Color('id', legend=alt.Legend(title="Countries")),
opacity=alt.condition(leg_selection, alt.value(1), alt.value(0.2))
).add_selection(leg_selection).properties(title='The ratio of deaths to confirmed cases (case fatality rate)', width=600)

upper = base.encode(
    alt.X('yearmonthdate(date):T',axis=alt.Axis(title='Date'),
          scale=alt.Scale(domain=brush))
)

lower = base.properties(
    height=60
).add_selection(brush)

upper & lower

alt.VConcatChart(...)